In step 1, I saved the molecules as RDKit object with 3D coordinates. This will enable the use of 3D descriptors like GETAWAY and various surface descriptors (VSA).
<p>
For generating features, I will re-use a PyTorch Dataset subclass I had written for a previous project. Given a list of molecules, it generates values for most descriptors implemented in RDKit on first acess to that molecule and cashes them for later reuse. Original version <a href="https://github.com/zetadin/TransferLearningFromPLS/blob/main/computeDescriptors.py">here</a>.

# Import the custom Dataset subclass

In [4]:
from tqdm import tqdm, trange

from CustomMolDataset import CustomMolDataset